<a href="https://colab.research.google.com/github/vivi-sobral/gemini/blob/main/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Import the Python SDK
import google.generativeai as genai
import pandas as pd
import numpy as np
# Used to securely store your API key
genai.configure(api_key="apikeyaqui")

generator_config = {
    "candidate_count": 1,
    "temperature": 0.6,
    "top_k" : 3,
    "top_p": 0.1
}

safety_seetings = {
    "HARASSMENT":"BLOCK_MEDIUM_AND_ABOVE",
    "HATE":"BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL":"BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS":"BLOCK_MEDIUM_AND_ABOVE"
}


In [6]:
title = "A proxima geração IA"
sample_text = ("tittulo: A próxima geração IA"
               "\n"
               "Artigo completo: "
               "\n"
               "GeminiApi: Uma forma de interagir com a IA facil e simples."
               )
for model in genai.list_models():
    print(model)


Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [8]:

embeddings = genai.embed_content(model="models/embedding-001", content=sample_text,title=title,task_type='RETRIEVAL_DOCUMENT')
print(embeddings)

{'embedding': [0.049727593, -0.02101213, -0.02632217, -0.0068139755, 0.07232266, -0.017138148, 0.0008381795, -0.05113851, 0.04581969, 0.062338036, -0.002270411, 0.012063966, -0.057974618, -0.015462862, -0.024904871, -0.0077813356, 0.01505253, -0.0075455224, -0.030562984, 0.0072914083, -0.038916726, 0.0086958585, -0.019524943, -0.03840701, 0.0012891621, 0.010125149, 0.01007787, -0.03404447, -0.02869886, -0.00044025914, -0.025781564, 0.034516416, -0.026149092, 0.012225588, -0.010740363, -0.054255113, -0.05636275, -0.0594688, 0.0045245253, 0.044590916, 0.01047384, -0.064071156, -0.01998215, 0.017922183, 0.0036248227, -0.05316276, 0.013268694, 0.038680762, 0.02490118, -0.06817053, 0.047066264, -0.0011466737, 0.09783028, -0.039135743, -0.0076628523, 0.013644595, 0.06524776, -0.037050605, 0.005617983, -0.0023270121, 0.02713506, 0.006421078, 0.019001491, 0.053381257, -0.004099712, -0.019155448, -0.020312222, 0.009109582, 0.034792617, 0.01231363, -0.008149294, -0.007738474, 0.0599119, -0.05781

In [16]:
document1={
    "titulo": "trocar pneu de qualquer carro",
    "conteudo": "chame o seguro que vai ser mais facil"
}
document2={
    "titulo": "limpar o estofado do veiculo",
    "conteudo": "é melhor nao tentar fazer voce mesmo"
}
document3={
    "titulo" : "qual melhor mecanico",
    "conteudo": "o que voce sempre vai"
}

documents = [document1,document2,document3]
df = pd.DataFrame(documents)
df


,titulo,conteudo
0,trocar pneu de qualquer carro,chame o seguro que vai ser mais facil
1,limpar o estofado do veiculo,é melhor nao tentar fazer voce mesmo
2,qual melhor mecanico,o que voce sempre vai


In [30]:
def embedd_fn(title,sample_text):
  return genai.embed_content(model="models/embedding-001", content=sample_text,title=title,task_type='RETRIEVAL_DOCUMENT')['embedding']

In [31]:
df['embeddings']=df.apply(lambda row: embedd_fn(row['titulo'],row['conteudo']),axis=1)

In [18]:
df

,titulo,conteudo,embeddings
0,trocar pneu de qualquer carro,chame o seguro que vai ser mais facil,"[0.016405078, -0.051943477, -0.046580464, -0.0..."
1,limpar o estofado do veiculo,é melhor nao tentar fazer voce mesmo,"[-0.03532933, -0.047468323, -0.04097805, -0.04..."
2,qual melhor mecanico,o que voce sempre vai,"[-0.00827313, 0.005335058, 0.01620885, -0.0138..."


In [36]:
def gerar_e_buscar_consulta(consulta,base,model):
    embedding_da_consulta = genai.embed_content(model=model,content=consulta,task_type="RETRIEVAL_QUERY")["embedding"]
    produtos_escalares = np.dot(np.stack(base["embeddings"]),embedding_da_consulta)
    indice=np.argmax(produtos_escalares)
    return base.iloc[indice]["conteudo"]

In [37]:
model = "models/embedding-001"
consulta = "Como faço para trocar um pneu de um goolge car?"
trecho=gerar_e_buscar_consulta(consulta,df,model)
trecho

'chame o seguro que vai ser mais facil'